In [22]:
cd code


E:\Major_project\code


In [23]:
import os
import torch
import torchvision
from torch import nn
import numpy as np
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from pathlib import Path
from timeit import default_timer as timer
from torch.utils.data import Subset


In [24]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)


device = "cuda" if torch.cuda.is_available() else "cpu"
device



'cpu'

In [5]:
pwd

'E:\\Major_project\\code'

In [41]:
class EnsembleModel(nn.Module):
    def __init__(self, model1, model2, model3):
        super(EnsembleModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        
    def forward(self, x):
        x = x.to(device)
        preds1 = torch.sigmoid(self.model1(x))
        preds2 = torch.sigmoid(self.model2(x))
        preds3 = torch.sigmoid(self.model3(x))
        preds = (preds1 + preds2 + preds3) / 3
        return preds # return the predictions on the same device as input

In [28]:
pwd

'E:\\Major_project\\code'

In [43]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.models import resnet18
from sklearn.metrics import confusion_matrix

# # Define the transform for the test data

# # Load the test dataset
# test_dir = 'path/to/test/data'
# test_dataset = datasets.ImageFolder(test_dir, transform=data_transform1)
data_transform = transforms.Compose([
    # Resize our images to 64x64
    transforms.Resize(size=(320, 320)),
    # Flip the images randomly on the horizontal 
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
    #transforms.RandomInvert(0.4),
    # Turn the image into a torch.Tensor
    transforms.ToTensor()
])

val_dir= 'Disease_vs_Normal/test'
test_dataset = datasets.ImageFolder(val_dir, transform=data_transform)

model = torch.load('models/Disease/ensemble_model (1).pt', map_location=torch.device('cpu'))

# Load the pretrained ResNet18 model
# model = torch.load("models/Disease_vs_Normal/Multiple_model1_25_04_2023.Resnet_V2_8-Batch_70.pt" , map_location=torch.device('cpu'))

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Evaluate the model on the test dataset
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for inputs, labels in test_dataset:
        inputs = inputs.to(device)
        labels = torch.tensor(labels).to(device)
        outputs = model(inputs.unsqueeze(0)).to(device)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend((labels.item(),))

# Compute the confusion matrix
cm = confusion_matrix(true_labels, predictions)

# Compute the accuracy
accuracy = (cm.diagonal().sum() / cm.sum())

print("Confusion matrix:")
print(cm)
print("Accuracy:", accuracy)

Confusion matrix:
[[215  73]
 [ 34 254]]
Accuracy: 0.8142361111111112


In [ ]:
import gradio as gr
import torch
import torchvision.models as models
from torchvision import transforms

# model = torch.load('E:/Major_project/code/models/Disease_Vs _Normal/super_model_disease_vs_normal.pt',map_location=torch.device('cpu'))
class_names= ['Disease', 'Normal']


def preprocess(image):
    image = image.resize((224,224))
    tensor = transforms.ToTensor()(image)
    tensor = tensor.unsqueeze(0)
    return tensor

def predict(image):
    tensor = preprocess(image)
    outputs = model(tensor)
    _, predicted = torch.max(outputs.data, 1)
    return class_names[predicted.item()]

interface = gr.Interface(
    fn=predict,
    inputs=gr.inputs.Image(type='pil', label='Input image'),
    outputs=gr.outputs.Label(num_top_classes=2, label='Predicted label')
)

interface.launch()